importamos las librerias

In [1]:

import pandas as pd
import sqlite3
import json
import requests


conectamos con la base de datos creada en sqlitebrowser

In [2]:
conn = sqlite3.connect("f2.db")

ahora ingesto la data en dataframes para revisarla

In [3]:
d1 = pd.read_csv("data/circuits.csv")
d2 = pd.read_csv("data/constructors.csv")
d3 = pd.read_csv("data/drivers.csv")
d4 = pd.read_csv("data/lap_times.csv")
d5 = pd.read_csv("data/pit_stops.csv")
d6 = pd.read_csv("data/qualifying.csv")
d7 = pd.read_csv("data/races.csv")
d8 = pd.read_csv("data/results.csv")

hacemos .info para ver datos faltantes

In [11]:
d8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25660 entries, 0 to 25659
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         25660 non-null  int64  
 1   raceId           25660 non-null  int64  
 2   driverId         25660 non-null  int64  
 3   constructorId    25660 non-null  int64  
 4   number           25660 non-null  object 
 5   grid             25660 non-null  int64  
 6   position         25660 non-null  object 
 7   positionText     25660 non-null  object 
 8   positionOrder    25660 non-null  int64  
 9   points           25660 non-null  float64
 10  laps             25660 non-null  int64  
 11  time             25660 non-null  object 
 12  milliseconds     25660 non-null  object 
 13  fastestLap       25660 non-null  object 
 14  rank             25660 non-null  object 
 15  fastestLapTime   25660 non-null  object 
 16  fastestLapSpeed  25660 non-null  object 
 17  statusId    

ahora llenamos las tablas

In [15]:
d1.to_sql("circuits", conn)
d2.to_sql("constructors", conn)
d3.to_sql("drivers", conn)
d4.to_sql("lap_times", conn)
d5.to_sql("pit_stops", conn)
d6.to_sql("qualifying", conn)
d7.to_sql("races", conn)
d8.to_sql("results", conn)


25660

establezco las relaciones y resuelvo otros detalles en sqlitebrowser

ahora comienzo con las querys (Luego las resuelvo con FastAPI)

1 Año con mas Carreras

In [5]:
year_max = pd.read_sql('select r.year as "Year", count(r.raceId) as "Races for Year" from races r group by "Year" order by "Races for Year" desc limit 2;', conn)
year_max


,Year,Races for Year
0,2022,22
1,2021,22


2. Piloto con mayor cantidad de primeros puestos

In [18]:
rawd_top = pd.read_sql('select ("forename" || " " || "surname") as "Name", count(r.driverId) as "Races Won" from drivers d, results r where r.driverId = d.driverId and r.position = 1 group by "Name" order by "Races Won" desc limit 1;', conn)
rawd_top

,Name,Races Won
0,Lewis Hamilton,103


3. Nombre del circuito mas corrido

In [21]:

cir_max = pd.read_sql('select c.name as "Circuit Name", count(r.circuitid) as "Races Hosted" from circuits c, races r where r.circuitid = c.circuitid group by "Circuit Name" order by "Races Hosted" desc limit 1;', conn)
cir_max

,Circuit Name,Races Hosted
0,Autodromo Nazionale di Monza,72


4. Piloto con mayor cantidad de puntos en total, cuyo constructor sea de nacionalidad sea American o British

In [6]:
piloto = pd.read_sql('select ("forename" || " " || "surname") as "Name", c.nationality as "Constructor Nationality", sum(r.points) as "Total Points" from drivers d, results r, constructors c  where r.driverId = d.driverId and r.constructorId = c.constructorId and (c.nationality = "British" or d.nationality = "American") group by "Name" order by "Total Points" desc limit 1;', conn)
piloto

,Name,Constructor Nationality,Total Points
0,Lewis Hamilton,British,6085.5


In [ ]:
circuits = pd.read_sql('select * from constructors;', conn)
circuits.to_json(orient="records")

In [23]:
# URL
url = 'http://127.0.0.1:8000/circuits'

# Get content
r = requests.get(url)

# Decode JSON response into a Python dict:
content = r.json()
pd.read_json(content)


,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park
...,...,...,...,...,...,...,...,...,...
72,73,BAK,Baku City Circuit,Baku,Azerbaijan,40.37250,49.85330,-7,http://en.wikipedia.org/wiki/Baku_City_Circuit
73,74,hanoi,Hanoi Street Circuit,Hanoi,Vietnam,21.01660,105.76600,9,http://en.wikipedia.org/wiki/Hanoi_Street_Circuit
74,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108,http://en.wikipedia.org/wiki/Algarve_Internati...
75,76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255,http://en.wikipedia.org/wiki/Mugello_Circuit
